In [1]:
# import libraries
import subprocess
import json
import os

from constants import *
from dotenv import load_dotenv
from pathlib import Path
from pprint import pprint
from web3 import Web3
from eth_account import Account
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.middleware import geth_poa_middleware
from web3.gas_strategies.time_based import medium_gas_price_strategy

load_dotenv()

True

In [2]:
# import mnemonic from env
filepath=".env"
p = subprocess.Popen(filepath, shell=True, stdout = subprocess.PIPE)

#get mnemonic
mnemonic = os.getenv('MNEMONIC')
print(mnemonic)


help effort false meadow walnut slice butter foil another feel minor wrap guide aunt tooth


In [3]:
# connect to local ETH/geth
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

In [4]:
# Define function to derive wallet
def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --cols=path,address,privkey,pubkey --format=json' 
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return  keys

In [5]:
# Test the function derive_wallets
derive_wallets(mnemonic,'BTC', 3)

[{'path': "m/44'/0'/0'/0/0",
  'address': '1MMbfLCwJc2ge5wxjfgtJav3Fac7gVnyTm',
  'privkey': 'L3bW4JZH2Vo4dyr8cmqH5K45jgVuqDz4gui4viJFhuhi2VXPrf7A',
  'pubkey': '03861bc6f1eec9f6231149b5070ea0fe5b7774b921ac9a0fbe5e8a505be51e6bf4'},
 {'path': "m/44'/0'/0'/0/1",
  'address': '1D2ydxj5YQsfYZkfyMYa3puUj77rW1h5a6',
  'privkey': 'L4cyrgYztUaX4J9WzJKbr8ss5azwPF3RAhck31cQopEp7fcZMKHA',
  'pubkey': '02cc54f8d4623cef5d62d1249e95ed888c07dcab593f046685a8ba758c2babe5f4'},
 {'path': "m/44'/0'/0'/0/2",
  'address': '17sSyfGjxjhd9kJvKddXobeVADqkjpnz17',
  'privkey': 'L4Ax23c48A1VtVMnSN1UiUVihNTousnzybF2GBmp1TrCaYRz2Dbb',
  'pubkey': '0240b64712342715b09a3a7ec8c02f190c77a17662456ecf2f8847f38e7c2acf55'}]

In [6]:
#Setting dictionary of coins to be used in the derive_wallets.
keys = {"btc", "btc-test", "eth"}
numderive = 3

coins = {}
for coin in keys:
    coins[coin]= derive_wallets(mnemonic, coin, numderive=3)

print(json.dumps(coins, indent=2, sort_keys=True))

{
  "btc": [
    {
      "address": "1MMbfLCwJc2ge5wxjfgtJav3Fac7gVnyTm",
      "path": "m/44'/0'/0'/0/0",
      "privkey": "L3bW4JZH2Vo4dyr8cmqH5K45jgVuqDz4gui4viJFhuhi2VXPrf7A",
      "pubkey": "03861bc6f1eec9f6231149b5070ea0fe5b7774b921ac9a0fbe5e8a505be51e6bf4"
    },
    {
      "address": "1D2ydxj5YQsfYZkfyMYa3puUj77rW1h5a6",
      "path": "m/44'/0'/0'/0/1",
      "privkey": "L4cyrgYztUaX4J9WzJKbr8ss5azwPF3RAhck31cQopEp7fcZMKHA",
      "pubkey": "02cc54f8d4623cef5d62d1249e95ed888c07dcab593f046685a8ba758c2babe5f4"
    },
    {
      "address": "17sSyfGjxjhd9kJvKddXobeVADqkjpnz17",
      "path": "m/44'/0'/0'/0/2",
      "privkey": "L4Ax23c48A1VtVMnSN1UiUVihNTousnzybF2GBmp1TrCaYRz2Dbb",
      "pubkey": "0240b64712342715b09a3a7ec8c02f190c77a17662456ecf2f8847f38e7c2acf55"
    }
  ],
  "btc-test": [
    {
      "address": "mrmvv1Z72hu1G1gXMwQBXBEJxge6K5qmEu",
      "path": "m/44'/1'/0'/0/0",
      "privkey": "cQTa9p3dxEv9vWHxQWjxK5TiEJehyKJjPd72L3gD8rZCNyHeCrF8",
      "pubkey": "0313bc

In [7]:
# select child accounts (and thus, private keys) by accessing items in the `coins` dictionary

eth_PrivateKey = coins["eth"][0]['privkey']
btc_PrivateKey = coins['btc-test'][0]['privkey']

print(json.dumps(eth_PrivateKey))
print(json.dumps(btc_PrivateKey))

"0x414ef66e83cf52ed7afced231fd1f02eb3d3c2194e566b769458f7b726aa0826"
"cQTa9p3dxEv9vWHxQWjxK5TiEJehyKJjPd72L3gD8rZCNyHeCrF8"


In [8]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.

def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

In [9]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.

def create_tx(coin, account, to, amount):
    if coin == ETH: 
        gasEstimate = w3.eth.estimateGas(
            {"from":account.address, "to":to, "value": amount}
        )
        return { 
            "from": account.address,
            "to": to,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
            #"chain_ID": w3.eth.chain_id
        }
    
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])


In [10]:
# call `create_tx`, sign the transaction, then send it to the designated network    

def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, to, amount)
        signed_tx = account.signTransaction(raw_tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()

    elif coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed_tx = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed_tx)    
    

In [11]:
# Send BTCTEST transaction using below code
btc_key = coins[BTCTEST][0]['privkey']
btc_address = coins[BTCTEST][1]['address']
send_tx(BTCTEST, priv_key_to_account(BTCTEST, btc_key),btc_address, 0.0000001)

In [13]:
# Send ETH transaction usinb below code
eth_key = coins[ETH][0]['privkey']
eth_address = coins[ETH][1]['address']
send_tx(ETH, priv_key_to_account(ETH, eth_key),eth_address, 1)


0x45070fac3ed43b2f3f1d7195e30e4dffc775d72b40d2690adf178198eb8a6450


'0x45070fac3ed43b2f3f1d7195e30e4dffc775d72b40d2690adf178198eb8a6450'